In [ ]:
# !pip install /kaggle/input/sci-llm-pip/bitsandbytes-0.40.0.post4-py3-none-any.whl

In [85]:

import pandas as pd

train_df = pd.read_csv('6000_train_examples.csv')
train_df=train_df.reset_index(drop=True).dropna()
train_df.head()

,prompt,A,B,C,D,E,answer
0,What is the primary role of Robin Juhkental in...,Robin Juhkental is the bassist of Malcolm Linc...,Robin Juhkental is the keyboardist of Malcolm ...,Robin Juhkental is the drummer of Malcolm Linc...,Robin Juhkental is the lead singer of Malcolm ...,Robin Juhkental is the lead guitarist of Malco...,D
1,Which of the following statements is true rega...,The theory of relativity only encompasses one ...,Special relativity explains the law of gravita...,The theory of relativity does not encompass an...,Special relativity applies to the cosmological...,General relativity only applies to the motion ...,D
2,In which country was the 1920 collection of co...,United States,Germany,Australia,France,England,E
3,What is one of the areas that Shimon Dovid Cow...,"Environmental conservation, opposing deforesta...","Homosexuality, looser abortion laws and volunt...","Freedom of speech, advocating for increased li...","Gun control, advocating for stricter regulatio...","Animal rights, opposing the use of animals for...",B
4,When did the Dirt Road Diaries Tour begin and ...,"February 17, 2014 - November 26, 2014","January 17, 2014 - October 26, 2014","February 17, 2013 - November 26, 2013","March 17, 2013 - December 26, 2013","January 17, 2013 - October 26, 2013",E


In [86]:

from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)

In [87]:
train_ds

Dataset({
    features: ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', '__index_level_0__'],
    num_rows: 5997
})

In [88]:
from transformers import AutoTokenizer,AutoModelForMultipleChoice, TrainingArguments, Trainer, BitsAndBytesConfig

# model_dir = '/kaggle/input/science-exam-trained-model-weights/run_2/'
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/mowgli/miniconda3/envs/textgen2/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [89]:
train_ds[2427]

{'prompt': "What was Karl-Otto Apel's specialization and philosophical approach?",
 'A': 'Karl-Otto Apel specialized in aesthetics and was considered an existentialist.',
 'B': 'Karl-Otto Apel specialized in political philosophy and was considered a communication theorist.',
 'C': 'Karl-Otto Apel specialized in moral philosophy and was considered a poststructuralist.',
 'D': 'Karl-Otto Apel specialized in the philosophy of language and was considered a phenomenologist.',
 'E': 'Karl-Otto Apel specialized in epistemology and was considered a critical theorist.',
 'answer': 'D',
 '__index_level_0__': 2428}

In [90]:
# Adapted from Huggingface Multiple Choice https://huggingface.co/docs/transformers/tasks/multiple_choice
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True) # tokenizer call using 'text_pair' which basically just adds a separator between the two sentences
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example

tokenized_train_ds = train_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

Map: 100%|██████████| 5997/5997 [00:02<00:00, 2012.08 examples/s]


In [91]:
# Following datacollator (adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice)
# will dynamically pad our questions at batch-time so we don't have to make every question the length
# of our longest question.

from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [92]:
from accelerate import Accelerator

In [93]:
current_device = Accelerator().process_index

In [94]:
current_device

0

In [95]:
# Now we'll instatiate the model that we'll finetune on our public dataset, then use to
# make prediction on the private dataset.
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForMultipleChoice.from_pretrained('microsoft/deberta-v3-base',device_map={"": current_device},torch_dtype=torch.float16)#,load_in_8bit=True)#,quantization_config=bnb_cfg)#load_in_4bit=True)

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
import peft

In [97]:
model=peft.prepare_model_for_int8_training(model)

/home/mowgli/miniconda3/envs/textgen2/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [106]:
# The arguments here are selected to run quickly; feel free to play with them.
checkpoint_dir = 'finetuned'
training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    evaluation_strategy="epoch",
#     save_strategy="steps",
#     load_best_model_at_end=True,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=40,
    weight_decay=0.01,
    report_to='none'
)

In [107]:
model

DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDi

In [111]:
# !pip install peft --no-index --find-links=file:///kaggle/input/llm-dl-training-libraries/peft
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["query_proj", "value_proj"],
    lora_dropout=0.05,
    bias="none",
    # fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS
)
peft_model = get_peft_model(model, 
                            lora_config)

In [112]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [113]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1179648
all model parameters: 185603330
percentage of trainable model parameters: 0.64%


In [114]:
# Generally it's a bad idea to validate on your training set, but because our training set
# for this problem is so small we're going to train on all our data.
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_train_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

In [115]:
torch.cuda.empty_cache() 

In [116]:
# Training should take about a minute
trainer.train()

/home/mowgli/miniconda3/envs/textgen2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.610400,1.608934
2,1.608700,1.606317
3,1.604600,1.556949
4,1.505600,1.473073
5,1.471100,1.440792
6,1.433400,1.388556
7,1.407400,1.360573
8,1.379000,1.340064
9,1.375900,1.321548
10,1.363300,1.312045


TrainOutput(global_step=30000, training_loss=1.3302001180013021, metrics={'train_runtime': 8871.5325, 'train_samples_per_second': 27.039, 'train_steps_per_second': 3.382, 'total_flos': 3.606854885005086e+16, 'train_loss': 1.3302001180013021, 'epoch': 40.0})

In [21]:
del model,peft_model

In [61]:
# Now we can actually make predictions on our questions
predictions = trainer.predict(tokenized_train_ds)

In [62]:
# The following function gets the indices of the highest scoring answers for each row
# and converts them back to our answer format (A, B, C, D, E)
import numpy as np
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)
    

In [63]:
# Let's double check our output looks correct:
predictions_to_map_output(predictions.predictions)

array(['A B C', 'A B C', 'A B C', ..., 'A B C', 'A B C', 'A B C'],
      dtype='<U5')

In [25]:
# Now we can load up our test set to use our model on!
# The public test.csv isn't the real dataset (it's actually just a copy of train.csv without the answer column)
# but it has the same format as the real test set, so using it is a good way to ensure our code will work when we submit.
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
test_df.head()

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


In [26]:
# There are more verbose/elegant ways of doing this, but if we give our test set a random `answer` column
# we can make predictions directly with our trainer.
test_df['answer'] = 'A'

# Other than that we'll preprocess it in the same way we preprocessed test.csv
test_ds = Dataset.from_pandas(test_df)
tokenized_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])


  0%|          | 0/200 [00:00<?, ?ex/s]

In [27]:
# Here we'll generate our "real" predictions on the test set
test_predictions = trainer.predict(tokenized_test_ds)

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [28]:
# Now we can create our submission using the id column from test.csv
submission_df = test_df[['id']]
submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)

submission_df.head()

/tmp/ipykernel_92/1317637749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)


,id,prediction
0,0,D C B
1,1,A C D
2,2,A C E
3,3,A C B
4,4,A D C


In [29]:
# Once we write our submission file we're good to submit!
submission_df.to_csv('submission.csv', index=False)